In [ ]:
# pip install langchain_community

In [ ]:
# pip install langchain_openai

In [3]:
# pip install Chroma

In [ ]:
# pip install chromadb

In [6]:
# pip install langchain-google-genai

In [7]:
import os

# Import classes from modules
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

In [8]:
# Generate API KEY from Gemini and OpenAI website and define as a variable.
os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEYS"
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEYS"

In [9]:
import json
from typing import overload
from langchain_core.documents import Document #type: ignore


@overload
def extract_data(path: str, main_key: str)->list[Document]:...

@overload
def extract_data(path: str)->list[Document]:...

def extract_data(path: str, main_key:str="items") -> list[Document]:
    documents: list[Document] = []
    with open(path, "r", encoding='utf8') as json_file:
        data:list[dict[str, str]] = json.load(json_file)[main_key]

        for each_data in data:
            page_content: str = ""
            for k in each_data:
                page_content = page_content + f"{k} {each_data[k]}"
            current_document = Document(
                page_content=page_content,
            )
            documents.append(current_document)
    return documents

In [10]:
json_file_path="fruit.json"
data = extract_data(json_file_path)

In [11]:
data

[Document(page_content='ชื่อ มะม่วงรสชาติ หวานสรรพคุณ ช่วยย่อยอาหาร, มีวิตามินซีสูงราคา 30 บาท/กิโลกรัม'),
 Document(page_content='ชื่อ ทุเรียนรสชาติ หวาน, มันสรรพคุณ ให้พลังงานสูง, มีวิตามินบีราคา 150 บาท/กิโลกรัม'),
 Document(page_content='ชื่อ ส้มรสชาติ หวาน, เปรี้ยวสรรพคุณ มีวิตามินซีสูง, ช่วยเสริมสร้างภูมิคุ้มกันราคา 60 บาท/กิโลกรัม'),
 Document(page_content='ชื่อ แอปเปิ้ลรสชาติ หวาน, กรอบสรรพคุณ ช่วยลดคอเลสเตอรอล, มีไฟเบอร์สูงราคา 80 บาท/กิโลกรัม'),
 Document(page_content='ชื่อ กล้วยรสชาติ หวานสรรพคุณ ช่วยให้พลังงาน, มีโพแทสเซียมสูงราคา 25 บาท/กิโลกรัม'),
 Document(page_content='ชื่อ สตรอว์เบอร์รีรสชาติ หวาน, เปรี้ยวสรรพคุณ มีวิตามินซีสูง, ช่วยบำรุงผิวราคา 120 บาท/กิโลกรัม'),
 Document(page_content='ชื่อ แตงโมรสชาติ หวาน, ฉ่ำสรรพคุณ ช่วยดับกระหาย, มีวิตามินเอราคา 20 บาท/กิโลกรัม'),
 Document(page_content='ชื่อ มังคุดรสชาติ หวาน, เปรี้ยวสรรพคุณ ช่วยลดการอักเสบ, มีสารต้านอนุมูลอิสระราคา 50 บาท/กิโลกรัม'),
 Document(page_content='ชื่อ เงาะรสชาติ หวาน, ฉ่ำสรรพคุณ ช่วยบำรุงผิว, มีวิตา

In [12]:
text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [13]:
texts

[Document(page_content='ชื่อ มะม่วงรสชาติ หวานสรรพคุณ ช่วยย่อยอาหาร, มีวิตามินซีสูงราคา 30 บาท/กิโลกรัม'),
 Document(page_content='ชื่อ ทุเรียนรสชาติ หวาน, มันสรรพคุณ ให้พลังงานสูง, มีวิตามินบีราคา 150 บาท/กิโลกรัม'),
 Document(page_content='ชื่อ ส้มรสชาติ หวาน, เปรี้ยวสรรพคุณ มีวิตามินซีสูง, ช่วยเสริมสร้างภูมิคุ้มกันราคา 60 บาท/กิโลกรัม'),
 Document(page_content='ชื่อ แอปเปิ้ลรสชาติ หวาน, กรอบสรรพคุณ ช่วยลดคอเลสเตอรอล, มีไฟเบอร์สูงราคา 80 บาท/กิโลกรัม'),
 Document(page_content='ชื่อ กล้วยรสชาติ หวานสรรพคุณ ช่วยให้พลังงาน, มีโพแทสเซียมสูงราคา 25 บาท/กิโลกรัม'),
 Document(page_content='ชื่อ สตรอว์เบอร์รีรสชาติ หวาน, เปรี้ยวสรรพคุณ มีวิตามินซีสูง, ช่วยบำรุงผิวราคา 120 บาท/กิโลกรัม'),
 Document(page_content='ชื่อ แตงโมรสชาติ หวาน, ฉ่ำสรรพคุณ ช่วยดับกระหาย, มีวิตามินเอราคา 20 บาท/กิโลกรัม'),
 Document(page_content='ชื่อ มังคุดรสชาติ หวาน, เปรี้ยวสรรพคุณ ช่วยลดการอักเสบ, มีสารต้านอนุมูลอิสระราคา 50 บาท/กิโลกรัม'),
 Document(page_content='ชื่อ เงาะรสชาติ หวาน, ฉ่ำสรรพคุณ ช่วยบำรุงผิว, มีวิตา

In [14]:
# This part is used for embedding the docs and storing them into Vector DB and initializing the retriever.
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

In [15]:
custom_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    You are an AI thai language model assistant.
    You are an expert at answering questions about fruits.
    Answer the question based ONLY on the following context.



    {context}
    Original question: {question}""",
)

In [16]:
# Create the language model
llm1 = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Create the chain
chain1 = ConversationalRetrievalChain.from_llm(
    llm=llm1,
    retriever=docsearch.as_retriever(),
    combine_docs_chain_kwargs={"prompt": custom_template}
)

In [17]:
chat_history = []
query = None  # Initialize query to avoid potential reference error

while True:
    if not query:
        query = input("User: ")
    if query in ['quit', 'q', 'exit']:
        break
    result = chain1.invoke({"question": query, "chat_history": chat_history})
    print("Chatbot:", result['answer'])

    chat_history.append((query, result['answer']))
    query = None

User: สรรพคุณของกล้วย
Chatbot: สรรพคุณของกล้วยคือ ช่วยให้พลังงาน และมีโพแทสเซียมสูง 

User: q
